<a href="https://colab.research.google.com/github/PondKann/KKU_data_mining/blob/master/Data_Online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Function

In [1]:
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/DataViz_2021'
os.listdir(path)

['pm-18-jan-2021.csv',
 'covid-19-daily_data_dictionary.xlsx',
 'income.csv',
 'income2.xls',
 'จัดซื้อ 57-58.xls',
 'online_retail_II.xlsx']

In [4]:
data_path = os.path.join(path,'online_retail_II.xlsx')

# Data1

In [ ]:
data1 = pd.read_excel(data_path,sheet_name='Year 2009-2010')

In [20]:
data1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [6]:
data1.count()

Invoice        525461
StockCode      525461
Description    522533
Quantity       525461
InvoiceDate    525461
Price          525461
Customer ID    417534
Country        525461
dtype: int64

## Clean Data

In [24]:
df1 = data1.dropna()

In [25]:
df1.dropna().count()

Invoice        417534
StockCode      417534
Description    417534
Quantity       417534
InvoiceDate    417534
Price          417534
Customer ID    417534
Country        417534
dtype: int64

In [8]:
size_before_drop = 525461
size_after_drop = 417534
print(f'size before drop = {size_before_drop} \nsize after drop = {size_after_drop}\nwhich is {100*((size_before_drop-size_after_drop)/size_before_drop)}')

size before drop = 525461 
size after drop = 417534
which is 20.53948818275762


In [26]:
df1.head( )

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [76]:
df11 = df1[['Invoice','StockCode']]
df11

,Invoice,StockCode
0,489434,85048
1,489434,79323P
2,489434,79323W
3,489434,22041
4,489434,21232
...,...,...
525456,538171,22271
525457,538171,22750
525458,538171,22751
525459,538171,20970


In [ ]:
df1

In [40]:
type({df1})

TypeError: ignored

In [68]:
table1 = df1.groupby('Invoice')['StockCode'].apply(list)
table1

Invoice
489434     [85048, 79323P, 79323W, 22041, 21232, 22064, 2...
489435                          [22350, 22349, 22195, 22353]
489436     [48173C, 21755, 21754, 84879, 22119, 22142, 22...
489437     [22143, 22145, 22130, 21364, 21360, 21351, 213...
489438     [21329, 21252, 21100, 21033, 20711, 21410, 214...
                                 ...                        
C538121                                              [22461]
C538122                                              [22444]
C538123                                [22331, 22333, 22956]
C538124                                    [M, 22699, 22423]
C538164                                             [35004B]
Name: StockCode, Length: 23587, dtype: object

In [74]:
table1 = str('Invoice')
table1

"{'Invoice'}"

In [36]:
!pip install apyori
from apyori import apriori

In [66]:
list(apriori(table1, min_support = 0.02 , min_confidence = 0.01 ))

TypeError: ignored

# Data2

In [ ]:
data2 = pd.read_excel(data_path,sheet_name='Year 2010-2011')

In [21]:
data2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
data2.count()

Invoice        541910
StockCode      541910
Description    540456
Quantity       541910
InvoiceDate    541910
Price          541910
Customer ID    406830
Country        541910
dtype: int64

In [11]:
data2.dropna().count()

Invoice        406830
StockCode      406830
Description    406830
Quantity       406830
InvoiceDate    406830
Price          406830
Customer ID    406830
Country        406830
dtype: int64

In [12]:
size_before_drop = 541910
size_after_drop = 406830
print(f'size before drop = {size_before_drop} \nsize after drop = {size_after_drop}\nwhich is {100*((size_before_drop-size_after_drop)/size_before_drop)}')

size before drop = 541910 
size after drop = 406830
which is 24.92664833643963


In [13]:
data1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [14]:
data2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [15]:
all_table = pd.concat([data1,data2])
all_table.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [17]:
!pip install apyori
from apyori import apriori

In [18]:
table = all_table.groupby('Invoice')['StockCode'].apply(list)
table

Invoice
489434     [85048, 79323P, 79323W, 22041, 21232, 22064, 2...
489435                          [22350, 22349, 22195, 22353]
489436     [48173C, 21755, 21754, 84879, 22119, 22142, 22...
489437     [22143, 22145, 22130, 21364, 21360, 21351, 213...
489438     [21329, 21252, 21100, 21033, 20711, 21410, 214...
                                 ...                        
C581484                                              [23843]
C581490                                       [22178, 23144]
C581499                                                  [M]
C581568                                              [21258]
C581569                                       [84978, 20979]
Name: StockCode, Length: 53628, dtype: object

In [19]:
list(apriori(table, min_support = 0.1 , min_confidence = 0.8 ))

TypeError: ignored